# Agent-Based Modeling with Mesa: Tutorial Notebook

This notebook provides a step-by-step guide to creating an agent-based model using the Mesa library. 
As we progress through the tutorial, we'll cover various aspects of creating, running, and analyzing agent-based models.

**Table of Contents:**

1. [Adding Space](#Adding-Space)
2. [Collecting Data](#Collecting-Data)
3. [Batch Run](#Batch-Run)

## Adding Space

In agent-based modeling (ABM), spatial elements are common, with agents traversing and interacting with neighboring entities. Mesa offers two primary spatial frameworks: grid and continuous space. Grids segment space into cells, restricting agents to specific locations akin to chess pieces on a board. In contrast, continuous space allows agents arbitrary positioning. Toroidal wrapping ensures grid continuity, preventing edge-related disparities. To introduce spatial dynamics, we'll situate agents on a grid, enabling random movement. Agents will transfer funds exclusively to those sharing their cell. Mesa provides various grid types, with MultiGrid enabling multiple agents per cell, facilitating our model's spatial representation and interactions.

In [ ]:
%%writefile starter_model/money_model.py
import mesa
from mesa.space import MultiGrid

class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other_agent = self.random.choice(cellmates)
            other_agent.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)
        # Create agents
        for i in range(self.num_agents):
            agent = MoneyAgent(i, self)
            self.schedule.add(agent)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

    def step(self):
        self.schedule.step()

In [ ]:
%%writefile starter_model/run.py
from money_model import MoneyModel
import numpy as np
import seaborn as sns

model = MoneyModel(100, 10, 10)
for i in range(20):
    model.step()
    
agent_counts = np.zeros((model.grid.width, model.grid.height))
for cell_content, (x, y) in model.grid.coord_iter():
    agent_count = len(cell_content)
    agent_counts[x][y] = agent_count

# Plot using seaborn, with a size of 5x5
g = sns.heatmap(agent_counts, cmap="viridis", annot=True, cbar=False, square=True)
g.figure.set_size_inches(4, 4)
g.set(title="Number of agents on each cell of the grid");

g.figure.savefig("output.png")

In [ ]:
!python -W ignore starter_model/run.py

## Collecting Data


Additionally, up to this point, obtaining data from the model required manual coding at the end of each simulation run, resulting in inefficiency and limited insights. To address this, Mesa introduces the DataCollector class, streamlining data collection and storage processes. This class categorizes data into model-level variables, agent-level variables, and tables, facilitating organized data management. By defining collection functions for model and agent-level variables, the DataCollector automatically captures relevant data during model execution. For instance, model-level collection functions operate on the model object, while agent-level functions operate on individual agents, associating collected values with the current model step. To demonstrate, we'll integrate a DataCollector into our model, capturing agent wealth at each step and calculating the model's Gini Coefficient to gauge wealth inequality.

In [ ]:
%%writefile starter_model/money_model.py
import mesa
from mesa.space import MultiGrid

def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B


class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        cellmates.pop(
            cellmates.index(self)
        )  # Ensure agent is not giving money to itself
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1
            if other == self:
                print("I JUST GAVE MONEY TO MYSELF HEHEHE!")

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini}, agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [ ]:
%%writefile starter_model/run.py
from money_model import MoneyModel
import numpy as np
import seaborn as sns

model = MoneyModel(100, 10, 10)
for i in range(10):
    model.step()
    
gini = model.datacollector.get_model_vars_dataframe()
# Plot the Gini coefficient over time
g = sns.lineplot(data=gini)
g.set(title="Gini Coefficient over Time", ylabel="Gini Coefficient");

g.figure.savefig("output.png")

In [ ]:
!python -W ignore starter_model/run.py

In [ ]:
%%writefile starter_model/run.py
from money_model import MoneyModel
import numpy as np
import seaborn as sns

model = MoneyModel(100, 10, 10)
for i in range(10):
    model.step()
    
agent_wealth = model.datacollector.get_agent_vars_dataframe()
    
agent_list = [3, 14, 25]

# Get the wealth of multiple agents over time
multiple_agents_wealth = agent_wealth[
    agent_wealth.index.get_level_values("AgentID").isin(agent_list)
]
# Plot the wealth of multiple agents over time
g = sns.lineplot(data=multiple_agents_wealth, x="Step", y="Wealth", hue="AgentID")
g.set(title="Wealth of agents 3, 14 and 25 over time");

g.figure.savefig("output.png")

In [ ]:
!python -W ignore starter_model/run.py

## Batch Run

Moreover, running a model just once is often insufficient; instead, you typically execute it multiple times with consistent parameters to discern overall data distributions. Additionally, varying parameters allow for the analysis of their impact on the model's outputs and behaviors. Instead of laboriously coding nested for-loops for each scenario, Mesa offers the batch_run function, automating this process. Furthermore, implementing the batch runner necessitates an additional variable, self.running, within the MoneyModel class. This variable enables the conditional shutdown of the model when specified criteria are met. For illustration purposes, we'll set it as True indefinitely in this example.

In [ ]:
%%writefile starter_model/money_model.py
import mesa
from mesa.space import MultiGrid

# Batch running the model
def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)
        self.running = True

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini},
            agent_reporters={"Wealth": "wealth", "Steps_not_given": "steps_not_given"},
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1
        self.steps_not_given = 0

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1
            self.steps_not_given = 0
        else:
            self.steps_not_given += 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()
        else:
            self.steps_not_given += 1

In [ ]:
%%writefile starter_model/run.py
from money_model import MoneyModel
import mesa
import numpy as np
import pandas as pd
import seaborn as sns

params = {"width": 10, "height": 10, "N": range(5, 100, 5)}

results = mesa.batch_run(
    MoneyModel,
    parameters=params,
    iterations=7,
    max_steps=100,
    number_processes=1,
    data_collection_period=1,
    display_progress=False,
)

results_df = pd.DataFrame(results)

# the agent choice is arbitrary and doesn't affect the results
results_filtered = results_df[(results_df.AgentID == 0) & (results_df.Step == 100)]

g = sns.pointplot(data=results_filtered, x="N", y="Gini", linestyle='none')
g.figure.set_size_inches(8, 4)
g.set(
    xlabel="Number of agents",
    ylabel="Gini coefficient",
    title="Gini coefficient vs. number of agents",
);

g.figure.savefig('output.png')

In [ ]:
!python -W ignore starter_model/run.py

In [ ]:
%%writefile starter_model/run.py
from money_model import MoneyModel
import mesa
import numpy as np
import pandas as pd
import seaborn as sns

params = {"width": 10, "height": 10, "N": [5,20,70,100]}

results = mesa.batch_run(
    MoneyModel,
    parameters=params,
    iterations=7,
    max_steps=100,
    number_processes=1,
    data_collection_period=1,
    display_progress=False,
)

results_df = pd.DataFrame(results)

# the agent choice is arbitrary and doesn't affect the results
results_filtered = results_df[results_df.AgentID == 0]

# Create a lineplot with error bars
g = sns.lineplot(
    data=results_filtered,
    x="Step",
    y="Gini",
    hue="N",
    errorbar=("ci", 95),
    palette="tab10",
)
g.figure.set_size_inches(8, 4)
plot_title = "Gini coefficient for different population sizes\n(mean over 100 runs, with 95% confidence interval)"
g.set(title=plot_title, ylabel="Gini coefficient");

g.figure.savefig('output.png')

In [ ]:
!python -W ignore starter_model/run.py